<a href="https://colab.research.google.com/github/ensarkaya/CS425-Group7-TermProject/blob/master/LASTVersion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import csv
import pandas as pd
from collections import defaultdict
import nltk
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
path = '/content/gdrive/My Drive/DataSets/100bin.csv'
df = pd.read_csv(path)
size =99946

In [45]:
nltk.download('stopwords')
dummy =["and","or","is","are","what","where","who","whom","when","in","cannot","with"
            ,"a","an","of","to","for","from","by","can","i","with","its","can't",
             "there","that","thats","all","does","do","did","the","am","using","it",
             "trying","how","be","not","doesn't","don't","didn't","won't","will",
             "my","be","should","would","as","use","must","have","but","however","at",
             "another","other","one","two","three","four","five","still","on","why","it"
              ,"how","a","to","'",".",",",";",":","?","!","-","_","1","2","3","4","5","6","7","8","9","0"]
stop = stopwords.words('english')
stop.append(dummy)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
for i in range(size):
  str1 = df.loc[i,'body']
  soup = BeautifulSoup(str1)
  for a in soup.find_all('p'):
    if a.string is None:
      df.loc[i,'body']="None"  
    else:
      df.loc[i,'body']=a.string

In [0]:
df["title"] = df["title"].apply(lambda x: x.lower())
df["title"] = df["title"].str.split(' ').apply(lambda x: ' '.join(k for k in x if k not in stop))
df["tags"] = df["tags"].apply(lambda x: x.lower())
df["tags"] = df["tags"].str.split('|').apply(lambda x: ' '.join(k for k in x if k not in dummy))
df["tags"] = df["tags"].str.split('-').apply(lambda x: ' '.join(k for k in x if k not in dummy))
#df["body"] = df["body"].str.split(' ').apply(lambda x: ' '.join(k for k in x if k not in stop))

In [0]:
#keywords -> id's of questions
dic = {}
#questionId -> (score, answerCount, viewCount, comment_count)
values = {}

for i in range(size):
  values[df.loc[i,'id']] = (df.loc[i,'title'], df.loc[i,'score'], df.loc[i,'answer_count'], df.loc[i,'view_count'], df.loc[i,'comment_count'])
  #print("idler : ",df.loc[i,'id'])
  #print("valueler : ",values[df.loc[i,'id']])
  df.loc[i,'title']#unneccessary
  string1 = df.loc[i,'title'].split()
  string2= df.loc[i,'tags'].split()
  #string3=df.loc[i,'body'].split()
  string=string1+string2#+string3
  for j in range(len(string)):
    while len(string[j]) > 0 and (string[j][0] < 'a' or string[j][0] > 'z') and (string[j][0] < '0' or string[j][0] > '9') and string[j][0] != '#' and string[j][0] != '+':
      string[j] = string[j][1:]
    while len(string[j]) > 0 and (string[j][len(string[j])-1] < 'a' or string[j][len(string[j])-1] > 'z') and (string[j][len(string[j])-1] < '0' or string[j][len(string[j])-1] > '9') and string[j][len(string[j])-1] != '#' and string[j][len(string[j])-1] != '+':
      string[j] = string[j][:len(string[j])-1]
      
    if  string[j] != "":
      if string[j] in dic:
        dic[string[j]].append(df.loc[i,'id'])
      else:
        dic[string[j]]=[df.loc[i,'id']]


In [0]:
dic2 = {} #keywords -> keyword count
dic3 = {} #questionId -> keywords if greater than threshold
dic4 = {} #keyword -> index
dic5 = {} #questionId -> keywords as indices

threshold = 2; 
count2=0
for i in dic:
#   print(i)
  count=0
  key = i
  
  for j in dic[i]:
    count+=1
  dic2[i]=count
#   print(i," **** ", count)
  if count > threshold:
    for j in range(count):
      id = dic[i][j] 
      if id in dic3:
        dic3[id].append(i)
      else:
        dic3[id] = [i]
    if key not in dic4:
      dic4[key] = count2
      count2+=1

In [0]:
for i in dic3:
  List = []
  for j in dic3[i]:
    if j in dic4:
      List.append(dic4[j])
  dic5[i] = List
        
# file = open("output.txt", "w")   
# for i in dic5:   
# #   print(i," : ",dic5[i])
# #   var = i + " : " + dic5[i]
#   file.write("%d" % i)
#   file.write(" : ")
#   for j in dic5[i]:
#     file.write("%d " % j)
#   file.write("\n")
# file.close()

In [0]:
import random

randomNum={}
for i in range(128):
  a=random.randint(1,50000)
  b=random.randint(1,50000)
  randomNum[i]=(a,b)

In [0]:
col=len(dic5)
row=128
matrix = [[0] * col for i in range(row)]
reverseId = {}
for i in range(128):
  count=0
  for j in dic5:
    minSoFar=9999999
    for k in dic5[j]:
      hashval=(randomNum[i][0]*k + randomNum[i][1]) % 50311
      if hashval<minSoFar:
        minSoFar = hashval
    matrix[i][count]=minSoFar
    #reverse[0] -> excel 0. satırdaki Q IDsi
    reverseId[count] = j
    count +=1  

In [0]:
file2 = open("signature.txt", "w")

for i in range(128):
    for j in range(size):
      file2.write("%d" % matrix[i][j])
      file2.write(" ")
    file2.write("\n")

In [56]:
#keywords
question = input()
s = question.split(" ")
string = [x for x in s if x not in dummy]

print(string)

for j in range(len(string)):
  string[j] = string[j].lower()
  while len(string[j]) > 0 and (string[j][0] < 'a' or string[j][0] > 'z') and (string[j][0] < '0' or string[j][0] > '9') and string[j][0] != '#' and string[j][0] != '+':
      string[j] = string[j][1:]
  while len(string[j]) > 0 and (string[j][len(string[j])-1] < 'a' or string[j][len(string[j])-1] > 'z') and (string[j][len(string[j])-1] < '0' or string[j][len(string[j])-1] > '9') and string[j][len(string[j])-1] != '#' and string[j][len(string[j])-1] != '+':
      string[j] = string[j][:len(string[j])-1]
      
      
print("final string : ", string)      

if(len(string) <= 0):
  print('Question not valid.\n')
else:
    #calculating the metascore
        
    #min-hash of the keywords
    indexQ = []
    for i in string:
      if i in dic4:
        indexQ.append(dic4[i]);
    print("index : ", indexQ)
    minHash = []
    for i in randomNum:
      minValue = 50311
      for j in indexQ:
        hashValue = (randomNum[i][0]*j + randomNum[i][1]) % 50311
        if(hashValue < minValue):
          minValue = hashValue
      minHash.append(minValue)
      
    #print("minHash list : ",minHash)
    
    #locality sensitive hashing
    orders=[]
    candidates = []
    r = 4
    band = 128 // r;
    
    
    print("band : ",band)
    
    index = 0;
    for i in range(band):
      string = ''
      for j in range(r):
        string = string + str(minHash[i*r + j])
        hashvalue = hash(string)
        #print("string check :", string, "\n")
        #print("hashvalue : ", hashvalue)
      for j in range(col):
        string = ''
        for k in range(r):
          string = string + str(matrix[i*r + k][j])
          if(hash(string) == hashvalue):
            if reverseId[j] not in candidates:
              candidates.append( reverseId[j] )
              orders.append(j)
              print("candidate question found ", reverseId[j])

    
    #output
    score = 0;
    answerCount = 0
    viewCount = 0
    commentCount = 0
    for id in candidates:
      #print("candidate id : ",id)
      score += values[id][1]
      answerCount += values[id][2]
      viewCount += values[id][3]
      commentCount += values[id][4]
    length = len(candidates)
    if (length == 0):
        print ("Completely new question!")
    else:
      print ("Predicted score: ", score/length, "\n")
      print ("Predicted number of answers: ", answerCount/length, "\n")
      print ("Predicted view count: ", viewCount/length, "\n")
      print ("Predicted number of comments: ", commentCount/length, "\n")

java compile error sytanx method array
['java', 'compile', 'error', 'sytanx', 'method', 'array']
final string :  ['java', 'compile', 'error', 'sytanx', 'method', 'array']
index :  [133, 1080, 95, 26, 380]
band :  32
candidate question found  50046478
candidate question found  49117854
candidate question found  49311229
candidate question found  48074248
candidate question found  49803528
candidate question found  50128318
candidate question found  52029031
candidate question found  48577383
candidate question found  51826880
candidate question found  51298491
candidate question found  51297100
candidate question found  52038308
candidate question found  51038181
candidate question found  49456009
candidate question found  51796787
candidate question found  49571375
candidate question found  50502123
candidate question found  50598689
candidate question found  51875336
candidate question found  52077683
candidate question found  50577038
candidate question found  49269157
candidate ques

In [64]:
    #Jaccard Similarity
    for x in range(len(candidates)):
      list1=[]
      list1.append(dic5[reverseId[orders[x]]]) 
      list2= indexQ
      intersection=0
      for y in range(len(list2)):
         if list2[y] in list1[0]:
          intersection=intersection+1
      union = (len(list1[0]) + len(list2)) - intersection
      sim = float(intersection / union)
      #print("list1: ", list1, " list2: ",list2)
      print("Sim= ",sim, " id: ",reverseId[x], "intersection:", intersection, "union: ", union)

Sim=  0.25  id:  51046313 intersection: 3 union:  12
Sim=  0.16666666666666666  id:  50327756 intersection: 2 union:  12
Sim=  0.18181818181818182  id:  50976577 intersection: 2 union:  11
Sim=  0.18181818181818182  id:  50312958 intersection: 2 union:  11
Sim=  0.25  id:  51986920 intersection: 2 union:  8
Sim=  0.2857142857142857  id:  50713895 intersection: 2 union:  7
Sim=  0.2857142857142857  id:  50407744 intersection: 2 union:  7
Sim=  0.3  id:  50708575 intersection: 3 union:  10
Sim=  0.2727272727272727  id:  51974391 intersection: 3 union:  11
Sim=  0.2  id:  51236943 intersection: 2 union:  10
Sim=  0.16666666666666666  id:  51105137 intersection: 2 union:  12
Sim=  0.25  id:  49652151 intersection: 2 union:  8
Sim=  0.2857142857142857  id:  48409973 intersection: 2 union:  7
Sim=  0.2857142857142857  id:  51893981 intersection: 2 union:  7
Sim=  0.2857142857142857  id:  49681082 intersection: 2 union:  7
Sim=  0.2222222222222222  id:  50702946 intersection: 2 union:  9
Sim=

In [58]:
print(dic4)

{'navigating': 0, 'pages': 1, 'php': 2, 'html': 3, 'sql': 4, 'mysqli': 5, 'assembly': 6, 'bootloader': 7, 'load': 8, 'gas': 9, 'low': 10, 'level': 11, 'regex': 12, 'counting': 13, 'characters': 14, 'two': 15, 'sides': 16, 'c#': 17, 'pointers': 18, 'std::string': 19, 'strange': 20, 'behavior': 21, 'c++': 22, 'string': 23, 'alamofire': 24, 'post': 25, 'method': 26, 'ios': 27, 'swift': 28, '4': 29, 'httprequest': 30, 'realloc': 31, 'failing': 32, 'second': 33, 'call': 34, 'c': 35, 'malloc': 36, 'calculate': 37, 'date': 38, 'given': 39, 'month': 40, 'week': 41, 'number': 42, 'within': 43, 'day': 44, 'javascript': 45, 'getting': 46, 'stored': 47, 'procedure': 48, 'result': 49, 'controller': 50, 'executes': 51, 'asp.net': 52, 'server': 53, 'entity': 54, 'framework': 55, 'class': 56, 'constructor': 57, 'called': 58, 'pointer': 59, 'memory': 60, 'management': 61, 'smart': 62, 'prevent': 63, 'google': 64, 'translate': 65, 'changing': 66, 'structure': 67, 'page': 68, 'css': 69, 'count': 70, 'inv